# Dings Trader — Full Pipeline (All-in-One)
Ein einziges Notebook für den kompletten Ablauf.

**Empfohlene Runtime-Strategie:**
- Schritte 0–3 auf **CPU/T4** (spart Credits)
- dann Runtime auf **A100** wechseln
- danach Schritte 4–7 ausführen

Alle Schritte schreiben Logs nach `logs/colab/full_pipeline.log` und drucken `OK:` / `ERROR:`.


## 0) Setup (Drive + Symlinks + Dependencies)


In [ ]:
step = "00_drive_mount"
try:
    from google.colab import drive
    drive.mount('/content/drive')
    print(f"OK: {step}")
except Exception as e:
    print(f"ERROR: {step} :: {e}")
    raise


In [ ]:
%%bash
    set -euo pipefail
    STEP="00_setup_symlinks_and_deps"
if [ -d "/content/Dings_Trader/TraderHimSelf" ]; then
  ROOT="/content/Dings_Trader/TraderHimSelf"
elif [ -d "/content/dings-trader/projects/dings-trader/TraderHimSelf" ]; then
  ROOT="/content/dings-trader/projects/dings-trader/TraderHimSelf"
else
  echo "ERROR: Repo nicht gefunden. Erst Clone/Update-Zelle ausführen."
  exit 1
fi
    cd "$ROOT"
    mkdir -p logs/colab
    LOG="logs/colab/full_pipeline.log"
    trap 'rc=$?; echo "ERROR: ${STEP} :: exit_code=${rc}" | tee -a "$LOG"; exit $rc' ERR

    echo "START: ${STEP}" | tee -a "$LOG"
    STORE="${STORE:-/content/drive/MyDrive/dings-trader-store/TraderHimSelf}"
mkdir -p "$STORE"/{data_raw,data_processed,models,logs,runs,checkpoints}

for d in data_raw data_processed models logs runs checkpoints; do
  [ -L "$d" ] && rm "$d" || true
  if [ -e "$d" ] && [ ! -L "$d" ]; then
    if [ -n "$(ls -A "$d" 2>/dev/null || true)" ]; then
      mv "$d" "${d}.backup.$(date +%s)"
    else
      rm -rf "$d"
    fi
  fi
  ln -s "$STORE/$d" "$d"
done

python -m pip install -r requirements.txt 2>&1 | tee -a "$LOG"
readlink -f data_raw | tee -a "$LOG"
    echo "OK: ${STEP}" | tee -a "$LOG"



## 1) Download Data (CPU/T4)


In [ ]:
%%bash
    set -euo pipefail
    STEP="01_download_data"
if [ -d "/content/Dings_Trader/TraderHimSelf" ]; then
  ROOT="/content/Dings_Trader/TraderHimSelf"
elif [ -d "/content/dings-trader/projects/dings-trader/TraderHimSelf" ]; then
  ROOT="/content/dings-trader/projects/dings-trader/TraderHimSelf"
else
  echo "ERROR: Repo nicht gefunden. Erst Clone/Update-Zelle ausführen."
  exit 1
fi
    cd "$ROOT"
    mkdir -p logs/colab
    LOG="logs/colab/full_pipeline.log"
    trap 'rc=$?; echo "ERROR: ${STEP} :: exit_code=${rc}" | tee -a "$LOG"; exit $rc' ERR

    echo "START: ${STEP}" | tee -a "$LOG"
    START_DATE="${START_DATE:-2019-01-01}"
END_DATE="${END_DATE:-$(date +%F)}"
SYMBOL="${SYMBOL:-BTCUSDT}"

python download_binance_data.py   --data-dir data_raw   --start-date "$START_DATE"   --end-date "$END_DATE"   --symbol "$SYMBOL" 2>&1 | tee -a "$LOG"

ls -lah data_raw | tee -a "$LOG"
    echo "OK: ${STEP}" | tee -a "$LOG"



## 2) Build Dataset (CPU/T4)


In [ ]:
%%bash
set -euo pipefail
STEP="02_build_dataset"
if [ -d "/content/Dings_Trader/TraderHimSelf" ]; then
  ROOT="/content/Dings_Trader/TraderHimSelf"
elif [ -d "/content/dings-trader/projects/dings-trader/TraderHimSelf" ]; then
  ROOT="/content/dings-trader/projects/dings-trader/TraderHimSelf"
else
  echo "ERROR: Repo nicht gefunden. Erst Clone/Update-Zelle ausführen."
  exit 1
fi
cd "$ROOT"
mkdir -p logs/colab
LOG="logs/colab/full_pipeline.log"
trap 'rc=$?; echo "ERROR: ${STEP} :: exit_code=${rc}" | tee -a "$LOG"; exit $rc' ERR

echo "START: ${STEP}" | tee -a "$LOG"
python build_dataset.py 2>&1 | tee -a "$LOG"
echo "OK: ${STEP}" | tee -a "$LOG"



## 3) Feature Engine (CPU/T4)


In [ ]:
%%bash
set -euo pipefail
STEP="03_feature_engine"
if [ -d "/content/Dings_Trader/TraderHimSelf" ]; then
  ROOT="/content/Dings_Trader/TraderHimSelf"
elif [ -d "/content/dings-trader/projects/dings-trader/TraderHimSelf" ]; then
  ROOT="/content/dings-trader/projects/dings-trader/TraderHimSelf"
else
  echo "ERROR: Repo nicht gefunden. Erst Clone/Update-Zelle ausführen."
  exit 1
fi
cd "$ROOT"
mkdir -p logs/colab
LOG="logs/colab/full_pipeline.log"
trap 'rc=$?; echo "ERROR: ${STEP} :: exit_code=${rc}" | tee -a "$LOG"; exit $rc' ERR

echo "START: ${STEP}" | tee -a "$LOG"
python feature_engine.py build 2>&1 | tee -a "$LOG"
echo "OK: ${STEP}" | tee -a "$LOG"



## 3.5) Runtime Switch auf A100
Jetzt Runtime auf **A100** wechseln (Runtime → Change runtime type).
Nach Reconnect die **nächsten zwei Zellen erneut** ausführen (Drive mount + setup), dann mit Schritt 4 weiter.


In [ ]:
step = "35_drive_remount_after_switch"
try:
    from google.colab import drive
    drive.mount('/content/drive')
    print(f"OK: {step}")
except Exception as e:
    print(f"ERROR: {step} :: {e}")
    raise


In [ ]:
%%bash
    set -euo pipefail
    STEP="35_relink_after_switch"
if [ -d "/content/Dings_Trader/TraderHimSelf" ]; then
  ROOT="/content/Dings_Trader/TraderHimSelf"
elif [ -d "/content/dings-trader/projects/dings-trader/TraderHimSelf" ]; then
  ROOT="/content/dings-trader/projects/dings-trader/TraderHimSelf"
else
  echo "ERROR: Repo nicht gefunden. Erst Clone/Update-Zelle ausführen."
  exit 1
fi
    cd "$ROOT"
    mkdir -p logs/colab
    LOG="logs/colab/full_pipeline.log"
    trap 'rc=$?; echo "ERROR: ${STEP} :: exit_code=${rc}" | tee -a "$LOG"; exit $rc' ERR

    echo "START: ${STEP}" | tee -a "$LOG"
    STORE="${STORE:-/content/drive/MyDrive/dings-trader-store/TraderHimSelf}"
cd "$ROOT"
for d in data_raw data_processed models logs runs checkpoints; do
  [ -L "$d" ] || { rm -rf "$d"; ln -s "$STORE/$d" "$d"; }
done
python -m pip install -r requirements.txt 2>&1 | tee -a "$LOG"
    echo "OK: ${STEP}" | tee -a "$LOG"



## 4) Train PatchTST (A100)


In [ ]:
%%bash
set -euo pipefail
STEP="04_train_patchtst"
if [ -d "/content/Dings_Trader/TraderHimSelf" ]; then
  ROOT="/content/Dings_Trader/TraderHimSelf"
elif [ -d "/content/dings-trader/projects/dings-trader/TraderHimSelf" ]; then
  ROOT="/content/dings-trader/projects/dings-trader/TraderHimSelf"
else
  echo "ERROR: Repo nicht gefunden. Erst Clone/Update-Zelle ausführen."
  exit 1
fi
cd "$ROOT"
mkdir -p logs/colab
LOG="logs/colab/full_pipeline.log"
trap 'rc=$?; echo "ERROR: ${STEP} :: exit_code=${rc}" | tee -a "$LOG"; exit $rc' ERR

echo "START: ${STEP}" | tee -a "$LOG"
python forecast/train_patchtst.py train --profile high-util --batch-size 1024 --num-workers 12 --prefetch-factor 4 --pin-memory true --persistent-workers true --amp true --compile true --log-every-batches 1 2>&1 | tee -a "$LOG"
echo "OK: ${STEP}" | tee -a "$LOG"




## 5) Precompute Forecast Features (A100)


In [ ]:
%%bash
set -euo pipefail
STEP="05_precompute_forecast"
if [ -d "/content/Dings_Trader/TraderHimSelf" ]; then
  ROOT="/content/Dings_Trader/TraderHimSelf"
elif [ -d "/content/dings-trader/projects/dings-trader/TraderHimSelf" ]; then
  ROOT="/content/dings-trader/projects/dings-trader/TraderHimSelf"
else
  echo "ERROR: Repo nicht gefunden. Erst Clone/Update-Zelle ausführen."
  exit 1
fi
cd "$ROOT"
mkdir -p logs/colab
LOG="logs/colab/full_pipeline.log"
trap 'rc=$?; echo "ERROR: ${STEP} :: exit_code=${rc}" | tee -a "$LOG"; exit $rc' ERR

echo "START: ${STEP}" | tee -a "$LOG"
# Hinweis: bei OOM zuerst --batch-size 768 oder 512 testen.
export PYTHONUNBUFFERED=1
stdbuf -oL -eL python -u forecast/train_patchtst.py precompute --profile high-util --batch-size 1024 --num-workers 12 --prefetch-factor 4 --pin-memory true --persistent-workers true --amp true --compile true --log-every-batches 10 2>&1 | tee -a "$LOG"
echo "OK: ${STEP}" | tee -a "$LOG"




## 6) Train PPO (A100)


In [ ]:
%%bash
set -euo pipefail
STEP="06_train_ppo"
if [ -d "/content/Dings_Trader/TraderHimSelf" ]; then
  ROOT="/content/Dings_Trader/TraderHimSelf"
elif [ -d "/content/dings-trader/projects/dings-trader/TraderHimSelf" ]; then
  ROOT="/content/dings-trader/projects/dings-trader/TraderHimSelf"
else
  echo "ERROR: Repo nicht gefunden. Erst Clone/Update-Zelle ausführen."
  exit 1
fi
cd "$ROOT"
mkdir -p logs/colab
LOG="logs/colab/full_pipeline.log"
trap 'rc=$?; echo "ERROR: ${STEP} :: exit_code=${rc}" | tee -a "$LOG"; exit $rc' ERR

echo "START: ${STEP}" | tee -a "$LOG"
python policy/train_ppo.py --profile high-util --n-steps 16384 --batch-size 1024 2>&1 | tee -a "$LOG"
echo "OK: ${STEP}" | tee -a "$LOG"



## 7) Final Report (copy/paste to chat)


In [ ]:
%%bash
    set -euo pipefail
    STEP="07_final_report"
if [ -d "/content/Dings_Trader/TraderHimSelf" ]; then
  ROOT="/content/Dings_Trader/TraderHimSelf"
elif [ -d "/content/dings-trader/projects/dings-trader/TraderHimSelf" ]; then
  ROOT="/content/dings-trader/projects/dings-trader/TraderHimSelf"
else
  echo "ERROR: Repo nicht gefunden. Erst Clone/Update-Zelle ausführen."
  exit 1
fi
    cd "$ROOT"
    mkdir -p logs/colab
    LOG="logs/colab/full_pipeline.log"
    trap 'rc=$?; echo "ERROR: ${STEP} :: exit_code=${rc}" | tee -a "$LOG"; exit $rc' ERR

    echo "START: ${STEP}" | tee -a "$LOG"
    echo "REPORT_START" | tee -a "$LOG"
python report_status.py 2>&1 | tee -a "$LOG"
echo "REPORT_END" | tee -a "$LOG"
    echo "OK: ${STEP}" | tee -a "$LOG"

